In [1]:
# Esse código pula alguns produtos mas extrái, anda pelas páginas e divide entre categorias e marcas.
import requests
from bs4 import BeautifulSoup
import time

# Função para criar a URL com filtro (categoria) e marca
def generate_url(filtro, brand, page_num):
    # Monta a URL com base na categoria (filtro) e marca
    url = f'https://www.drogariasminasmais.com.br/medicamentos/{filtro}/{brand}?initialMap=c&initialQuery=medicamentos&map=category-1,category-2,brand'
    # Adiciona a paginação
    return f'{url}&page={page_num}'

# Função para buscar os produtos de uma categoria e marca específica
def fetch_products_for_category_and_brand(filtro, brand):
    all_products = []  # Lista para armazenar todos os produtos encontrados
    page_num = 1  # Iniciar com a primeira página

    while True:
        # Gerar a URL para a página atual com o filtro e marca
        url = generate_url(filtro, brand, page_num)
        response = requests.get(url)
        
        if response.status_code != 200:
            print(f"Erro ao acessar a página {page_num}: {response.status_code}")
            break
        
        # Usando BeautifulSoup para parsear o conteúdo
        soup = BeautifulSoup(response.content, 'html.parser')

        # Encontrando os elementos dos produtos na página
        product_name_elements = soup.find_all('span', class_='vtex-product-summary-2-x-productBrand')
        price_elements = soup.find_all('span', class_='vtex-product-price-1-x-sellingPriceValue')

        # Se não houver produtos, paramos a execução
        if not product_name_elements:
            print(f'Não foram encontrados produtos na página {page_num}.')
            break
        
        # Exibindo produtos e preços
        for name, price in zip(product_name_elements, price_elements):
            product_name = name.get_text(strip=True)
            price_value = price.get_text(strip=True)
            all_products.append({'name': product_name, 'price': price_value})
            print(f'Produto: {product_name} - Preço: {price_value}')
        
        # Verificando se há mais páginas
        page_num += 1

    return all_products

# Função principal para buscar produtos por categoria (filtro) e todas as marcas
def search_products_for_category(filtro, brands):
    print(f'\nBuscando produtos para a categoria: {filtro}')
    
    all_category_products = []  # Lista para armazenar todos os produtos de uma categoria
    
    # Loop para buscar todos os produtos de cada marca dentro da categoria
    for brand in brands:
        print(f'\nBuscando produtos para a marca: {brand} na categoria: {filtro}')
        products = fetch_products_for_category_and_brand(filtro, brand)
        all_category_products.extend(products)  # Adiciona os produtos dessa marca à lista geral
        print(f'Total de produtos encontrados para a categoria {filtro} - marca {brand}: {len(products)}')
    
    print(f'\nTotal de produtos encontrados para a categoria {filtro}: {len(all_category_products)}')

# Exemplos de filtros (categorias) e marcas
filtros = [
    'mais-medicamentos', 'vitaminas', 'calmantes', 'gripe-e-resfriado', 'analgésicos', 'similar', 'hipertensão', 'anti-inflamatórios', 
    'colesterol-e-triglicerídeos', 'anticoncepcionais', 'alergias e infecções', 'fitoterápicos', 'genéricos', 'dor-e-febre', 'digestivo', 
    'relaxante-muscular', 'diabetes', 'gastrite', 'emagrecedores', 'disfunção-erétil'
]

brands = [
    'ems', 'ache-rx', 'eurofarma-rx', 'ems-generico', 'sigma-pharma-rx', 'eurofarma-generico', 'cimed', 'biolab-rx', 'libbs', 'germed-generico',
    'apsen', 'medley-genericos', 'momenta', 'supera', 'farmoquimica', 'myralis', 'prati-donaduzzi-generico', 'pharlab', 'abbott-rx', 'torrent', 
    'uniao-quimica-genom', 'ache-otc', 'cristalia-latinofarma', 'teuto-generico', 'asepxia', 'acnase', 'sanofi-rx', 'neo-quimica-generico', 'biolab-generico', 
    'gsk-otc', 'novartis-rx', 'zanphy', 'marjan', 'neo-quimica-similar', 'procter-gambl', 'gsk-rx', 'nova-quimica-generico', 'merck-rx', 
    'airela', 'servier', 'allergan', 'uniao-quimica-generico', 'alcon', 'biolab-avert-otc', 'geolab-generico', 'zodiac-adium', 'sankyo-pharma', 
    'cellera', 'teuto-similar', 'divcom', 'bayer-rx', 'astrazeneca', 'legrand-generico', 'herbarium', 'biosintetica-generico', 'sigma-pharma-otc', 
    'megalabs', 'boehringer', 'bayer-otc', 'brace-pharma-rx', 'hertz-p-2', 'takeda', 'globo-generico', 'reckitt-benckiser-rb', 'cifarma', 'ranbaxy',
    'natulab', 'biosintetica', 'aspen', 'zambon', 'arovida', 'arese', 'viatris', 'sandoz-generico', 'pfizer', 'gross', 'ems-similar', 'lundbeck', 'jj-otc', 
    'cristalia', 'belfar', 'massime', 'hertz-p-1', 'geolab', 'galderma', 'cifarma-similar', 'chiesi-farmalab', 'janssen', 'feelclean', 'elseve', 'althaia', 
    'grunenthal', 'baldacci', 'abv', 'uniao-quimica', 'theraskin', 'besins', 'exeltis', 'torrent-generico', 'merck-generico', 'maxinutri', 'm-s-d', 
    'hebron-quesalon', 'gummy', 'delta', 'bausch-lomb', 'vitamedic-generico', 'nikkho-zydus', 'lilly', 'leopharma', 'pharmascience', 'medquimica-generico', 
    'diffucap-chemobras', 'usk-rx', 'multilab', 'moksha8', 'medquimica', 'm-s-d-organon', 'catarinense', 'boiron', 'nativita-generico', 'globo', 'astellas', 
    'unichems', 'hisamitsu', 'health-labs', 'gsk-stiefel', 'farmax', 'eurofarma-oaz', 'abbott-nutricional', 'weleda', 'vitafor', 'ucb-biopharma', 'sandoz-rx', 
    'bago', 'waldemiro', 'valeant', 'osorio-de-moraes', 'mundipharma', 'medley-rx', 'legrand-rx', 'arte-nativa', 'zydus-generico', 'wyeth-rx', 'vitamedic', 
    'united-rx', 'roche', 'natcofarma', 'caninha', 'blau', 'shire', 'prati-donaduzzi', 'glenmark', 'genomma', 'equaliv', 'diversos-similares', 'biolab-similar', 
    'bio-soft', 'almaroni', 'teva', 'philips-avente', 'organon', 'mylan', 'imec', 'herbamed', 'farmace-generico', 'farmabraz-beta-atalaia-ltda', 'eli-lilly', 
    'cifarma-generico', 'casa-granado', 'umuarama', 'tresemm', 'trb', 'semina', 'sanitall', 'pequenin', 'olvebra', 'novartis-otc', 'nova-quimica-similar', 
    'nestle-industrial', 'nativita-similar', 'minancora', 'millet-roux', 'melpoejo', 'melora', 'latinofarma', 'laboratorios-ferring', 'johnson-johnson-s-a', 
    'glenmark-generico', 'gigita', 'curatec', 'cheez-it', 'cazi', 'bunker', 'brasterapica', 'biomm', 'bianco', 'almeida-prado-ltda', 'abl-brasil'

]

# Executar a busca para uma categoria específica
category_to_search = 'mais-medicamentos'  # Escolha a categoria que deseja buscar
search_products_for_category(category_to_search, brands)



Buscando produtos para a categoria: mais-medicamentos

Buscando produtos para a marca: ems na categoria: mais-medicamentos
Produto: Aglitil 30mg com 60 Comprimidos - Ems - Preço: R$111,74
Produto: Aglitil 15mg com 30 Comprimidos - Ems - Preço: R$65,70
Produto: Nesina Met 12,5 + 1000mg com 60 Comprimidos - Hypera - Preço: R$154,41
Produto: Nesina Pio 25mg + 30mg com 30 Comprimidos - Hypera - Preço: R$220,90
Produto: Nesina Pio 25mg + 15mg com 30 Comprimidos - Hypera - Preço: R$223,07
Produto: Nesina Met 12,5 + 850mg com 60 Comprimidos - Hypera - Preço: R$154,41
Produto: Nesina 12,5mg com 30 Comprimidos - Hypera - Preço: R$136,73
Produto: Maxsulid com 30 Comprimidos - Hypera - Preço: R$100,22
Produto: Predsim 20mg com 20 Comprimidos - Hypera - Preço: R$61,62
Produto: Predsim 40mg com 10 Comprimidos - Hypera - Preço: R$76,86
Produto: Gelol Gel Massageador - 60g - Preço: R$32,17
Produto: Tamarine Fibras Kids Sabor Morango - 240ml - Preço: R$71,13
Produto: Addera D3 5.000ui com 30 Comprimi

In [31]:
import requests
from bs4 import BeautifulSoup
import json
import re

# Fazendo a requisição ao site
url = 'https://www.drogariasminasmais.com.br/medicamentos'
headers = {
    "User-Agent": "Mozilla/5.0"  # Simula um navegador para evitar bloqueios
}
requisicao = requests.get(url, headers=headers)

# Criando o parser do BeautifulSoup
soup = BeautifulSoup(requisicao.text, 'html.parser')

# Buscando todos os scripts na página
scripts = soup.find_all('script')

# Usando while para encontrar o índice correto do script
i = 0
json_file = None
while i < len(scripts):
    script_text = scripts[i].text
    if '__STATE__' in script_text:
        print(f"Encontrado __STATE__ no script de índice {i}")  # Debug: verifica qual script contém a variável
        
        # Tentando extrair o JSON de forma mais robusta
        match = re.search(r'__STATE__\s*=\s*(\{.*\})', script_text, re.DOTALL)
        if match:
            try:
                json_file = json.loads(match.group(1))  # Converte string para JSON
                print("JSON encontrado e carregado com sucesso!")
                break
            except json.JSONDecodeError:
                print("Erro ao decodificar o JSON.")
                break
    i += 1

# Exibir os dados encontrados
if json_file:
    print(json.dumps(json_file, indent=4))  # Formata para leitura
else:
    print("Não foi possível encontrar o JSON com a variável __STATE__.")


Encontrado __STATE__ no script de índice 11
JSON encontrado e carregado com sucesso!
{
    "Product:sp-30959": {
        "advertisement": null,
        "cacheId": "sp-30959",
        "productId": "30959",
        "description": "Aglitil\u00ae\u00a0cont\u00e9m uma subst\u00e2ncia chamada\u00a0cloridrato de pioglitazona.Aglitil\u00ae\u00a0\u00e9 um medicamento antidiab\u00e9tico indicado juntamente com dieta e exerc\u00edcios f\u00edsicos para melhorar o controle da glicemia (n\u00edveis de a\u00e7\u00facar no sangue) em pacientes com\u00a0diabetes\u00a0mellitus\u00a0tipo II (n\u00e3o insulino dependente). Este \u00e9 um tipo de diabetes que usualmente se desenvolve na vida adulta.",
        "productName": "Aglitil 30mg com 60 Comprimidos - Ems",
        "productReference": "37048",
        "linkText": "37048",
        "brand": "EMS",
        "brandId": 699,
        "link": "/37048/p",
        "categories": {
            "type": "json",
            "json": [
                "/Medicamento